<a href="https://colab.research.google.com/github/aydinmyilmaz/Transformers-Tutorials/blob/master/sentiment_fine_tuning_huggingface_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### https://www.youtube.com/watch?v=VlW7WJi51Xk&ab_channel=AndrejBaranovskij

In [1]:
!pip install -qqq transformers

     |████████████████████████████████| 3.8 MB 5.5 MB/s 
     |████████████████████████████████| 6.5 MB 27.5 MB/s 
     |████████████████████████████████| 67 kB 1.9 MB/s 
     |████████████████████████████████| 895 kB 30.1 MB/s 
     |████████████████████████████████| 596 kB 29.5 MB/s 


In [2]:
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /content/sarcasm.json

--2022-04-03 17:14:45--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/content/sarcasm.json’

/content/sarcasm.js 100%[===================>]   5.38M  --.-KB/s    in 0.05s   

2022-04-03 17:14:45 (100 MB/s) - ‘/content/sarcasm.json’ saved [5643545/5643545]



In [4]:
training_size = 20000

with open("/content/sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
validation_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
validation_labels = labels[training_size:]

In [12]:
training_sentences[0:5]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way']

In [5]:
print(len(training_sentences))
print(len(validation_sentences))

20000
6709


In [6]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [7]:
train_encodings = tokenizer(training_sentences,
                            truncation=True,
                            padding=True,
                            return_tensors='tf')
val_encodings = tokenizer(validation_sentences,
                          truncation=True,
                          padding=True,
                          return_tensors='tf')

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

In [10]:
train_dataset.

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.list_files>

In [13]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [15]:
test_sentence = "former versace store clerk sues over secret 'black code' for minority shoppers"
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

[0.4904019 0.5095981]
[0.4770314  0.52296853]


In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [17]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=3,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/3
1250/1250 [==============================] - 354s 275ms/step - loss: 0.2650 - accuracy: 0.8868 - val_loss: 0.2053 - val_accuracy: 0.9133
Epoch 2/3
1250/1250 [==============================] - 340s 272ms/step - loss: 0.0925 - accuracy: 0.9661 - val_loss: 0.3167 - val_accuracy: 0.9168
Epoch 3/3
1250/1250 [==============================] - 340s 272ms/step - loss: 0.0396 - accuracy: 0.9873 - val_loss: 0.3093 - val_accuracy: 0.9167


In [18]:
model.save_pretrained("/content/sentiment_custom_model")

In [ ]:
#### Load saved model and run predict function

In [19]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("/content/sentiment_custom_model")

Some layers from the model checkpoint at /content/sentiment_custom_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/sentiment_custom_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [21]:
test_sentence ="former versace store clerk sues over secret 'black code' for minority shoppers"
test_sentence_sarcasm = "Amphibious pitcher makes debut"

predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")
tf_output = loaded_model.predict(predict_input)[0]
tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
print(tf_prediction)

predict_input_s = tokenizer.encode(test_sentence_sarcasm,
                                   truncation=True,
                                   padding=True,
                                   return_tensors="tf")
tf_output_s = model.predict(predict_input_s)[0]
tf_prediction_s = tf.nn.softmax(tf_output_s, axis=1).numpy()[0]
print(tf_prediction_s)

[0.996700 0.003300]
[0.000430 0.999570]
